In [2]:
"""
Dashboard Application for Animal Shelter Records

This script sets up a dashboard application using Dash and JupyterDash to interact with animal shelter records.
It connects to a MongoDB database using the CRUD Python module and displays data through various widgets.

Key Components:
1. Data Manipulation / Model
   - Connects to the MongoDB database using credentials.
   - Reads data from the database and prepares it for display.

2. Dashboard Layout / View
   - Defines the layout and visual elements of the dashboard.
   - Includes a header, interactive filtering options, a DataTable, graphs, and a map.

3. Interaction Between Components / Controller
   - Defines callback functions to update the DataTable, graphs, and map based on user interactions.
   - Handles MongoDB queries and updates the dashboard components accordingly.

Dependencies:
- dash
- dash_leaflet
- jupyter_dash
- plotly
- pandas
- numpy
- base64

Usage:
- Run the script in a Jupyter Notebook environment.
- Interact with the dashboard to filter data, view records, and visualize information on the map.

Developed by: Hannah Rose Morgenstein
Date: 08-18-2024
"""
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import numpy as np

# Import the CRUD Python module and class
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Define your database credentials
username = "aacuser"
password = "hrm"

# Try to connect to the database using the CRUD Python module
try:
    db = AnimalShelter(username, password)
except Exception as e:
    print(f"Error connecting to the database: {e}")
    db = None  # Set db to None if connection fails

# Try to read all records from the database and create a DataFrame
try:
    df = pd.DataFrame.from_records(db.read({})) if db else pd.DataFrame()
    if not df.empty:
        df.drop(columns=['_id'], inplace=True)  # Drop the MongoDB _id column as it's not needed
except Exception as e:
    print(f"Error fetching data: {e}")
    df = pd.DataFrame()  # Empty DataFrame as fallback

#########################
# Dashboard Layout / View
#########################
# Initialize the Dash app for Jupyter
app = JupyterDash(__name__)

# Load and encode the Grazioso Salvare logo image for embedding in the app
image_filename = 'Grazioso_Salvare_Logo.png'  # Path to your logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()  # Encode image in base64

# Define the layout of the app
# Define the layout of the app
app.layout = html.Div([
    # Header section at the top of the layout
    html.Div([
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Hr(),
        html.H2('Animal Shelter Dashboard'),
        html.P("Welcome to the CS-340 Dashboard. Use the filter options below to view animal data based on rescue type. Select a row in the DataTable to view details on the map."),
        html.Hr()
    ], style={'padding': '20px', 'backgroundColor': '#f2f2f2'}),
    
    # Header with the developer's unique identifier
    html.Header("Developer: Hannah Rose Morgenstein"),
    
    # Display the Grazioso Salvare logo with a clickable link to the client's home page
    html.A(
        href='https://www.snhu.edu',  # The URL to link to
        children=html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '300px'}),
        target='_blank'  # This opens the link in a new tab
    ),
    html.Hr(),
    
    # RadioItems for interactive filtering options
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',  # Default value is 'reset'
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    
    # DataTable to display animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],  # Column definitions
        data=df.to_dict('records'),  # Convert DataFrame to dictionary format for DataTable
        row_selectable='single',  # Allow selection of a single row
        page_size=10,  # Number of rows per page
        sort_action='native',  # Enable native sorting
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling
        style_cell={'textAlign': 'left'},  # Align text to the left
    ),
    html.Br(),
    html.Hr(),
    
    # Container for the pie chart and map side by side
    html.Div([
        # Pie chart
        html.Div(dcc.Graph(id='pie-chart'), style={'width': '48%', 'display': 'inline-block'}),
        
        # Map
        html.Div(id='map-id', style={'width': '48%', 'display': 'inline-block', 'height': '500px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),
    
    # Bar graph below the pie chart and map
    html.Div(dcc.Graph(id='bar-graph'), style={'width': '100%'}),
])

#############################################
# Interaction Between Components / Controller
#############################################
def get_query(filter_type):
    """
    Generate a MongoDB query based on the selected filter type.
    
    Args:
    - filter_type (str): The type of rescue operation selected by the user.
    
    Returns:
    - dict: MongoDB query dictionary based on filter type.
    """
    if filter_type == 'water':
        return {'$and': [{'animal_type': 'Dog'}, {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}}, {'sex_upon_outcome': 'Intact Female'}, {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}]}
    elif filter_type == 'mountain':
        return {'$and': [{'animal_type': 'Dog'}, {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}, {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}]}
    elif filter_type == 'disaster':
        return {'$and': [{'animal_type': 'Dog'}, {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}, {'sex_upon_outcome': 'Intact Male'}, {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}]}
    else:
        return {}  # Return empty query if no filter is selected

# Define a default map to be displayed when no specific location is selected
default_map = dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.2672, -97.7431], zoom=10, children=[
    dl.TileLayer(id="base-layer-id")
])

@app.callback(
    [Output('datatable-id', 'data'),
     Output('pie-chart', 'figure'),
     Output('map-id', 'children'),
     Output('bar-graph', 'figure')],
    [Input('filter-type', 'value'),
     Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_all_widgets(filter_type, viewData, selected_rows):
    """
    Update all dashboard widgets based on user input and data.
    
    Args:
    - filter_type (str): Selected filter type.
    - viewData (list): Data displayed in the DataTable.
    - selected_rows (list): Indices of selected rows in the DataTable.
    
    Returns:
    - list: Data for DataTable.
    - dict: Figure for pie chart.
    - list: Children for map component.
    - dict: Figure for bar graph.
    """
    try:
        if db is None:
            raise ConnectionError("Database connection is not available.")
        
        # Fetch data based on filter type
        if filter_type == 'reset':
            df = pd.DataFrame.from_records(db.read({}))
            df.drop(columns=['_id'], inplace=True, errors='ignore')
        else:
            query = get_query(filter_type)
            records = db.read(query)
            df = pd.DataFrame.from_records(records).drop(columns=['_id'], errors='ignore')
    except Exception as e:
        print(f"Error fetching data: {e}")
        df = pd.DataFrame()  # Empty DataFrame as fallback
        error_message = "An error occurred while fetching the data. Please try again later."
        return [], go.Figure(), [default_map], go.Figure()

    # Display an error message if no data is available
    if df.empty:
        error_message = "No data available or an error occurred."
        return [], go.Figure(), [default_map], go.Figure()

    # Create pie and bar charts
    pie_fig = px.pie(df, names='breed', title='Preferred Animals')
    bar_fig = px.bar(df, x='outcome_type', title='Outcome Types')

    # Handle DataTable selections and location-based map updates
    if viewData is None or selected_rows is None or len(viewData) == 0:
        return df.to_dict('records'), pie_fig, [default_map], bar_fig

    dff = pd.DataFrame.from_dict(viewData)

    if not selected_rows:
        return df.to_dict('records'), pie_fig, [default_map], bar_fig

    row = selected_rows[0]

    # Ensure location columns are present before creating the map
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return df.to_dict('records'), pie_fig, [default_map], bar_fig

    latitude = dff.iloc[row]['location_lat']
    longitude = dff.iloc[row]['location_long']
    breed = dff.iloc[row]['breed']
    animal_name = dff.iloc[row]['name']
    
    # Create a map showing the selected animal's location
    map_fig = dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[latitude, longitude], children=[
            dl.Tooltip(breed),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(animal_name)
            ])
        ])
    ])
    
    return df.to_dict('records'), pie_fig, [map_fig], bar_fig


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """
    Update the styling of DataTable columns based on selection.
    
    Args:
    - selected_columns (list): List of selected column IDs.
    
    Returns:
    - list: List of style dictionaries for DataTable columns.
    """
    if selected_columns is None or len(selected_columns) == 0:
        return []
    
    # Apply conditional styling to selected columns
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Run the app with debug mode enabled for development
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8574/
